In [1]:
%matplotlib inline

# import cvxpy as cp
import numpy as np

import scipy.optimize as opt

import random

import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [79]:
def rotate(x, y, angle):
    return [x * np.cos(np.radians(angle)) - y * np.sin(np.radians(angle)), 
            x * np.sin(np.radians(angle)) + y * np.cos(np.radians(angle))]

***States Setup***

In [115]:
# runway config

zero_heading = 270

runway_end_x = 2421 
runway_end_y = -1737

runway_start_x = 0
runway_start_y = 0

runway_heading = 54.331

runway_width = 100
runway_extend_length = 50

# sampling config

ws_bound = 5
wh_bound = 30

***Rotate Runway to Zero Heading***

In [116]:
ex, ey = rotate(runway_end_x, runway_end_y, -(runway_heading + zero_heading - 360))
sx, sy = rotate(runway_start_x, runway_start_y, -(runway_heading + zero_heading - 360))
ex, ey, sx, sy = int(np.floor(ex)), int(np.floor(ey)), int(np.floor(sx)), int(np.floor(sy))

sx -= runway_extend_length

runway_llx, runway_lly = sx, sy - runway_width
runway_urx, runway_ury = ex, ey + runway_width

(runway_llx, runway_lly), (runway_urx, runway_ury)

((-50, -100), (2979, 100))

***Discretize Runway & Apply Rotation Back***

In [114]:
def sample_environments(real_ws, real_wh, speed_bounds, heading_bounds, num_samples):
    env = [(real_ws, real_wh + zero_heading)]
    for _ in range(num_samples):
        env.append((np.random.randint(real_ws - speed_bounds, real_ws + speed_bounds + 1),
                   np.random.randint(real_wh - heading_bounds, real_wh + heading_bounds + 1) + zero_heading))
    return env


def generate_control_lookup(runway_x, runway_y, velocity_bins=(0, 100, 2), wind_speed=(0, 10, 1), 
                            wind_heading=(-40, 40, 5), box_dim=(10, 10)):
    runway_bot, runway_top = runway_y
    stride_x, stride_y = box_dim

    assert runway_right > runway_left, "right side of runway must be numerically greater than left side"
    assert runway_top > runway_bot, "top side of runway must be numerically greater than bottom side"
    
    leftover_x = stride_x - (runway_right - runway_left) % stride_x
    leftover_y = stride_y - (runway_top - runway_bot) % stride_y

    grid_search = {}
    lookup_table = {}
    grid_no = 0
    
    for i in range(runway_left, runway_right + leftover_x, stride_x):
        for j in range(runway_bot, runway_top + leftover_y, stride_y):
            center_x, center_y = i + stride_x / 2, j + stride_y / 2
            local_controls = {}
            # ITERATE THROUGH ALL COMBINATIONS OF INIT VELOCITY, INIT HEADING, WINDSPEEDS, WINDHEADINGS
            for h in range(0, 360):
                for v in range(velocity_bins[0], velocity_bins[1], velocity_bins[2]):
                    for ws in range(wind_speed[0], wind_speed[1], wind_speed[2]):
                        for wh in range(wind_heading[0], wind_heading[1], wind_heading[2]):
                            state0 = [center_x, center_y, v, h]
                            sample_environments()
                            # call solver here!
                            controls, _, _ = solve_states(new_init_states, desired_states, winds, plane_specs, acceleration_constraint,
                                  turning_constraint, time_step=time_step, sim_time=num_steps)
                            local_controls[(v, h, ws, wh)] = controls
            lookup_table[grid_no] = local_controls
            # GENERATE GRID POINTS LOOK UP
            for k in range(stride_x):
                for l in range(stride_y):
                    grid_search[(i + k, j + l)] = grid_no
            grid_no += 1
            
    return grid_search, lookup_table



    
    
grid_labels, control_table = discretize((runway_llx, runway_urx), (runway_lly, runway_ury), 
                                        box_dim=(5, 5))

In [ ]:
def rotate_centers():
    
    

***Rotate All Points Back***

In [87]:
# get bottom left & upper right coordinates

# runway_llx, runway_lly = rotate(sx, sy + runway_width, (runway_heading + zero_heading - 360))
# runway_urx, runway_ury = rotate(ex, ey - runway_width, (runway_heading + zero_heading - 360))
# runway_lrx, runway_lry = rotate(sx, sy - runway_width, (runway_heading + zero_heading - 360))
# runway_ulx, runway_uly = rotate(ex, ey + runway_width, (runway_heading + zero_heading - 360))